Demostration of `fmslib` in combination with discrete event simulation in `SimPy`

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys, os

In [2]:
sys.path.append('..')

In [3]:
import simpy
import random
from fsmlib import FSM, get_random_next_event, parse_transition_func

## SimPy example with Python generators

In [4]:
def get_process(env, name, duration_func):
    
    def p_gen(env):
        while True:
            
            t = duration_func()
            print('[{0}] {1}_start'.format(env.now, name))
            yield env.timeout(t)
            print('[{0}] {1}_stop'.format(env.now, name))
            
    p = env.process(p_gen(env))
    return p
    

In [5]:
env1 = simpy.Environment()

p1 = get_process(env1, 'p1', lambda: 5)
p2 = get_process(env1, 'p2', lambda: 5 + random.random())

env1.run(until=20)


[0] p1_start
[0] p2_start
[5] p1_stop
[5] p1_start
[5.462763980373588] p2_stop
[5.462763980373588] p2_start
[10] p1_stop
[10] p1_start
[11.411369878300102] p2_stop
[11.411369878300102] p2_start
[15] p1_stop
[15] p1_start
[16.91863003143355] p2_stop
[16.91863003143355] p2_start


## SimPy example with a generator inside a class and an `FSM` object

In [6]:
class SimpleProcess:
    '''
    An independent process with the defined duration function
    '''

    def __init__(self, name, env, duration_func, fsm):
        
        self.name = name
        self.env = env
        self.duration_func = duration_func
        self.fsm = fsm
        
        self.stats = {'total_time': 0}
        
        self.action = env.process(self.p_gen())
    
    def p_gen(self):
        
        while True:
            
            next_event = get_random_next_event(self.fsm)
            next_state = self.fsm.transition_func[(self.fsm.current_state, next_event)]
            
            t = self.duration_func[next_state]()
            self.stats['total_time'] += t
            
            self.fsm.advance(next_event)
            
            print('now={0:.4f}\t {1}:{2}!->{3} (will take {4})'.format(self.env.now, self.name, next_event, self.fsm.current_state, t))
            yield self.env.timeout(t)
            
    def print_event(self, event_name):
        print('[{0}] {1}_{2}'.format(self.env.now, self.name, event_name))

In [7]:
env2 = simpy.Environment()

simple_fsm = FSM('idle', {('idle', 'start'): 'running', ('running', 'stop'): 'idle'})

p1 = SimpleProcess('p1', env2, {'idle': lambda: 1, 'running': lambda: 5}, simple_fsm)
p2 = SimpleProcess('p2', env2, {'idle': lambda: 1, 'running': lambda: 5 + random.random()}, simple_fsm)

env2.run(until=50)

now=0.0000	 p1:start!->running (will take 5)
now=0.0000	 p2:stop!->idle (will take 1)
now=1.0000	 p2:start!->running (will take 5.3360928585173)
now=5.0000	 p1:stop!->idle (will take 1)
now=6.0000	 p1:start!->running (will take 5)
now=6.3361	 p2:stop!->idle (will take 1)
now=7.3361	 p2:start!->running (will take 5.068513508279528)
now=11.0000	 p1:stop!->idle (will take 1)
now=12.0000	 p1:start!->running (will take 5)
now=12.4046	 p2:stop!->idle (will take 1)
now=13.4046	 p2:start!->running (will take 5.638551790977366)
now=17.0000	 p1:stop!->idle (will take 1)
now=18.0000	 p1:start!->running (will take 5)
now=19.0432	 p2:stop!->idle (will take 1)
now=20.0432	 p2:start!->running (will take 5.205986330142965)
now=23.0000	 p1:stop!->idle (will take 1)
now=24.0000	 p1:start!->running (will take 5)
now=25.2491	 p2:stop!->idle (will take 1)
now=26.2491	 p2:start!->running (will take 5.619401617324347)
now=29.0000	 p1:stop!->idle (will take 1)
now=30.0000	 p1:start!->running (will take 5)
now

In [8]:
p1.stats, p2.stats

({'total_time': 53}, {'total_time': 51.360731748285026})

## Example with resources

In [9]:
class ResourceUser(SimpleProcess):
    def process_cycle(self):
        print('Hello')
        super(ResourceUser, self).process_cycle()

In [10]:
env = simpy.Environment()
rsc = simpy.Resource(env, capacity=1)

In [11]:
# TODO ...